In [1]:
import pandas as pd
import string
from num2words import num2words
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import PorterStemmer, WordNetLemmatizer

### Data Preprocessing

In [2]:
abc_data = pd.read_csv('../abc_reduced_data.csv')

# Remove numbers and symbols from the text
abc_data['headline_text'] = abc_data['headline_text'].str.replace('\d+', '', regex=True)
abc_data['headline_text'] = abc_data['headline_text'].str.translate(str.maketrans('', '', string.punctuation))

# Convert 'Sentence' column to lowercase text
text_lowercase = abc_data['headline_text'].str.lower()

# Tokenize the dataset
data_tokenized = text_lowercase.apply(word_tokenize)

# Remove stopwords from the dataset
stop_words_list = set(stopwords.words('english'))
stop_words = data_tokenized.apply(lambda x: [word for word in x if word not in stop_words_list])

# Perform stemming and lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
root_words = stop_words.apply(lambda x: [stemmer.stem(lemmatizer.lemmatize(word)) for word in x])

# Add ngrams to the dataset
data_ngrams = root_words.apply(lambda x: list(ngrams(x, 3)))

# Remove rows with empty lists of tuples 
data_nulls = data_ngrams.apply(lambda x: x if len(x) > 0 else None)

# Combine pre-processed data with original DataFrame
preprocessed_data = pd.concat([abc_data['publish_date'], data_nulls], axis=1)

# Remove any rows with null values
preprocessed_data = preprocessed_data.dropna()

preprocessed_data.to_csv("../1_Preprocessed_Data/abc_preprocessed.csv", index=False)
preprocessed_data.head()

publish_date                                      headline_text
0   2019-01-20  [(thousand, attend, woman), (attend, woman, ma...
1   2019-01-22  [(former, socceroo, craig), (socceroo, craig, ...
2   2019-01-14  [(medium, attack, outsid), (attack, outsid, co...
3   2019-01-28  [(ambul, offic, dy), (offic, dy, attend), (dy,...
4   2019-01-25  [(florida, offici, resign), (offici, resign, h...